In [3]:
import sys
import numpy as np
import cv2


src = cv2.imread('rose.bmp', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

cv2.imshow('src', src)

for sigma in range(1, 6):
    blurred = cv2.GaussianBlur(src, (0, 0), sigma)

    alpha = 1.0
    dst = cv2.addWeighted(src, 1 + alpha, blurred, -alpha, 0.0)

    desc = "sigma: %d" % sigma
    cv2.putText(dst, desc, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
               1.0, 255, 1, cv2.LINE_AA)

    cv2.imshow('dst', dst)
    cv2.waitKey()

cv2.destroyAllWindows()


In [ ]:
import numpy as np
import cv2


def sobel_derivative():
    src = cv2.imread(r'C:\ex\opencv_python\ch09\lenna.bmp', cv2.IMREAD_GRAYSCALE)

    if src is None:
        print('Image load failed!')
        return

    mx = np.array([[-1, 0, 1],
                   [-2, 0, 2],
                   [-1, 0, 1]], dtype=np.float32)
    my = np.array([[-1, -2, -1],
                   [0, 0, 0],
                   [1, 2, 1]], dtype=np.float32)

    dx = cv2.filter2D(src, -1, mx, delta=128)
    dy = cv2.filter2D(src, -1, my, delta=128)

    cv2.imshow('src', src)
    cv2.imshow('dx', dx)
    cv2.imshow('dy', dy)
    cv2.waitKey()
    cv2.destroyAllWindows()


def sobel_edge():
    src = cv2.imread('lenna.bmp', cv2.IMREAD_GRAYSCALE)

    if src is None:
        print('Image load failed!')
        return

    dx = cv2.Sobel(src, cv2.CV_32F, 1, 0)
    dy = cv2.Sobel(src, cv2.CV_32F, 0, 1)

    fmag = cv2.magnitude(dx, dy)
    mag = np.uint8(np.clip(fmag, 0, 255))
    _, edge = cv2.threshold(mag, 150, 255, cv2.THRESH_BINARY)

    cv2.imshow('src', src)
    cv2.imshow('mag', mag)
    cv2.imshow('edge', edge)
    cv2.waitKey()
    cv2.destroyAllWindows()


def canny_edge():
    src = cv2.imread(r'C:\ex\opencv_python\ch09\lenna.bmp', cv2.IMREAD_GRAYSCALE)

    if src is None:
        print('Image load failed!')
        return

    dst1 = cv2.Canny(src, 50, 100)
    dst2 = cv2.Canny(src, 50, 150)

    cv2.imshow('src', src)
    cv2.imshow('dst1', dst1)
    cv2.imshow('dst2', dst2)
    cv2.waitKey()
    cv2.destroyAllWindows()


if __name__ == '__main__':
    sobel_derivative()
    sobel_edge()
    canny_edge()


In [1]:
# 캐니 엣지를 활용해 진품 가품 구별하기

import cv2

def canny_edge():
    src = cv2.imread(r'C:\ex\cvdir2\img\watch.jpg', cv2.IMREAD_GRAYSCALE)

    if src is None:
        print('Image load failed!')
        return

    # threshold1의 값이 작아질수록 엣지 검출이 많아진다
    dst1 = cv2.Canny(src, 380, 100)
    dst2 = cv2.Canny(src, 10, 100)
    dst3 = cv2.Canny(src, 20, 100)

    cv2.imshow('src', src)
    cv2.imshow('dst1', dst1)
    cv2.imshow('dst2', dst2)
    cv2.imshow('dst3', dst3)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
if __name__ == '__main__':
    canny_edge()

Image load failed!


In [6]:
# 깨진 달걀 찾기

import cv2

def canny_edge():
    src = cv2.imread(r'C:\ex\cvdir\img\egg_detect.jpg', cv2.IMREAD_GRAYSCALE)

    if src is None:
        print('Image load failed!')
        return

    # threshold1의 값이 작아질수록 엣지 검출이 많아진다
    dst1 = cv2.Canny(src, 380, 100)
    dst2 = cv2.Canny(src, 10, 100)
    dst3 = cv2.Canny(src, 20, 100)

    cv2.imshow('src', src)
    cv2.imshow('dst1', dst1)
    cv2.imshow('dst2', dst2)
    cv2.imshow('dst3', dst3)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
if __name__ == '__main__':
    canny_edge()

In [27]:
# 이미지에서 달걀 검출
# 각 달걀의 내부에서만 엣지 검출을 수행
# 깨진 달걀을 찾아낸다

import cv2
import numpy as np

def canny_edge_in_circle():
    src = cv2.imread(r'C:\ex\cvdir\img\egg_detect.jpg', cv2.IMREAD_GRAYSCALE)
    # IMREAD_GRAYSCALE
    # 캐니 알고리즘이 일반적으로 단일 채널 이미지에서 작동하기 때문
    # 밝기 정보만을 담고 있어 엣지 검출에 적합
    
    if src is None:
        print('Image load failed!')
        return

    # 허프 변환을 이용해 원 검출
    blurred = cv2.blur(src, (3, 3))
    # 블러 처리를 통해 노이즈를 줄여서 검출의 정확성을 높인다
    circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, 1, 50,
                               param1=100, param2=30, # 원 검출의 임계값 조절
                               minRadius=15, maxRadius=95) # 원의 최소 및 최대 반지름 설정

    # 원 내부에서만 Canny Edge 검출을 하기 위한 준비
    dst = np.zeros_like(src)
    # 처음에 원본 이미지와 동일한 크기의 빈 이미지,
    # 즉 모든 값이 0인 이미지로 초기화

    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        for (cx, cy, radius) in circles:
            # 원 내부에 대한 마스크 생성
            mask = np.zeros_like(src)
            cv2.circle(mask, (cx, cy), radius, 255, -1)

            # 마스크된 영역에서만 Canny Edge 검출
            edges = cv2.Canny(src, 30, 200)
            dst = cv2.bitwise_or(dst, cv2.bitwise_and(edges, edges, mask=mask))

            # 검출된 원 표시
            cv2.circle(src, (cx, cy), radius, (255, 0, 0), 2, cv2.LINE_AA)

    cv2.imshow('src', src)
    cv2.imshow('dst', dst)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
if __name__ == '__main__':
    canny_edge_in_circle()